In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.preprocessing import scale
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

import time
import math

In [2]:
df = pd.read_pickle("../data/large200analysis.pickle")

In [3]:
df = df.loc[df["metadata.isAsymmetric"] == False]

In [4]:
# Drop NA columns
df = df.drop("complexFeatures.entropyDegreeDistribution", axis=1)
df = df.drop("complexFeatures.vertexParticipationCoefficient", axis=1)

In [5]:
import re
columnNames = list(df)
regexTimes = re.compile(".*Times")
timesColumnNames = list(filter(regexTimes.match, columnNames))
for column in timesColumnNames:
    columnNames.remove(column)
    
regexCosts = re.compile("heuristics.*Costs")
costsColumnNames = list(filter(regexCosts.match, columnNames))
for column in costsColumnNames:
    columnNames.remove(column)

columnNames.remove("metadata.isAsymmetric")
columnNames.remove("generated")
columnNames.remove("deepWalk")
columnNames.remove("sequenceLength")
columnNames.remove("costs")

In [6]:
# Drop rows with NA
rowsBefore = df.shape[0]
df = df.dropna()
print("Dropped %d rows due to None values" % (rowsBefore - df.shape[0]))

costValues = df[["heuristics.tabuCosts", "heuristics.simulatedAnnealingCosts", "heuristics.graspCosts", "heuristics.geneticCosts", "heuristics.antColonyCosts"]].values
indexRankings = costValues.argsort()

# Remove all *Times columns
df = df[columnNames]

# Remove name column
df = df.drop(["name"], axis=1)

Dropped 0 rows due to None values


In [7]:
size = df.shape[0]
# Test data is separated in cleaning stage
trainSize = int(size * 0.75)
validSize = size - trainSize

inputs = scale(df.astype('float64'),axis=1)

inputsTrain = inputs[0:trainSize]
outputsTrain = indexRankings[0:trainSize]

inputsValid = inputs[trainSize:]
outputsValid = indexRankings[trainSize:]

/home/gastinad/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [8]:
inputsValid.shape

(2155, 37)

In [9]:
outputsTrain

array([[2, 0, 4, 1, 3],
       [2, 0, 4, 1, 3],
       [2, 0, 4, 1, 3],
       ..., 
       [2, 4, 3, 1, 0],
       [2, 4, 3, 1, 0],
       [2, 4, 3, 1, 0]])

In [10]:
EPOCHS = 1000

N1 = trainSize
FEATURE_COUNT = df.shape[1]
LABEL_COUNT = 5

NODES1 = 512
NODES2 = 512
NODES3 = 256
NODES4 = 256
NODES5 = 128
NODES6 = 128
NODES7 = 64

ALPHA = 0.001

BATCH_SIZE = 2

STD = 0.1

LEARNING_RATE = 0.00001

In [11]:
# Define the input function for training
inputFunc = tf.estimator.inputs.numpy_input_fn(
    x={"input": inputsTrain.astype(np.float32)}, y=outputsTrain.astype(np.float32),
#     batch_size=BATCH_SIZE, num_epochs=EPOCHS, shuffle=True)
    num_epochs=EPOCHS, shuffle=True)

In [12]:
# Define the neural network
def network(xDict, mode):
    x = xDict["input"]
        
    regularizer = tf.contrib.layers.l2_regularizer(scale=ALPHA)
    
    layer1 = tf.layers.dense(x, NODES1, kernel_regularizer=regularizer, activation=tf.nn.relu)
    
    layer2 = tf.layers.dense(layer1, NODES2, activation=tf.nn.relu)
    layer3 = tf.layers.dense(layer2, NODES3, activation=tf.nn.relu)
    layer4 = tf.layers.dense(layer3, NODES4, activation=tf.nn.relu)
    layer5 = tf.layers.dense(layer4, NODES5, activation=tf.nn.relu)
    layer6 = tf.layers.dense(layer5, NODES6, activation=tf.nn.relu)
    layer7 = tf.layers.dense(layer6, NODES7, activation=tf.nn.relu)
    # Output fully connected layer with a neuron for each class
    outLayer = tf.layers.dense(layer7, LABEL_COUNT)
    return outLayer

# Loss Functions

In [13]:
def listNetLoss(label, prediction):
    softMaxLabel = tf.nn.softmax(label)
    softMaxPrediction = tf.nn.softmax(prediction)
    return -tf.reduce_mean(softMaxLabel * tf.log(softMaxPrediction))

def log2(x):
    numerator = tf.log(x)
    denominator = tf.log(tf.constant(2, dtype=numerator.dtype))
    return numerator / denominator


# Builds an integer ranking out of a 1-D tensor
def convertPredToRank(prediction):
    return tf.cast(tf.nn.top_k(prediction, k=5).indices, dtype=tf.float32)

# Accuracy Measures

In [14]:
# Accuracy metric using Normalized Discounted Cumulative Gain, as per https://github.com/shiba24/learning2rank/
def ndcg(labels, predictions, k=5):
    topK = tf.nn.top_k(labels, k=5)
    sortedValues = topK.values
    sortedIndices = topK.indices
#         print(labelSorted)
#         labelSorted = sorted(label, reverse=True)
    ideal_dcg = 0
    for i in range(k):
#             ideal_dcg += (2 ** labelSorted[:i] - 1.) / log2(tf.cast(i + 2, tf.float64))
        ideal_dcg += (tf.cast(sortedValues[i] + 1, tf.float32)) / log2(tf.cast(i + 2, tf.float32))
    dcg = 0
#         argsort_indices = np.argsort(predictions)[::-1]
#         argsort_indices = tf.nn.top_k(predictions, k=5).indices
#         print(argsort_indices)
    for i in range(k):
        dcg += (tf.gather(predictions, sortedIndices[i]) + 1) / log2(tf.cast(i + 2, tf.float32))
#         dcg += (predictions[i] + 1) / log2(tf.cast(i + 2, tf.float64))
    return dcg / ideal_dcg

def spearmanCorrelation(label, prediction):
    length = tf.cast(tf.shape(prediction)[0], tf.float32)
    sumVal = tf.reduce_sum(tf.square(tf.subtract(prediction, label)))
    return 1 - 6 * sumVal / (length ** 3 - length)

# Bound Spearman coeff. between 0 and 1
def boundedSpearman(label, prediction):
    return (spearmanCorrelation(label, prediction) + 1.)/2

def top1Match(label, prediction):
    return tf.cast(tf.equal(label[0], prediction[0]), tf.float32)

def top2Match(label, prediction):
    sameFirstOrSecond = tf.logical_or(tf.equal(label[0], prediction[0]), tf.equal(label[1], prediction[1]))
    sameFirstAndSecond = tf.logical_or(tf.equal(label[1], prediction[0]), tf.equal(label[0], prediction[1]))
    return tf.cast(tf.logical_or(sameFirstOrSecond, sameFirstAndSecond), tf.float32)

In [15]:
# Define the model function (following TF Estimator Template)
def modelFunc(features, labels, mode):
    # Build the neural network
    logits = network(features, mode)
    
#     resizedLogits = tf.reshape(logits, shape=[-1, MAX_SIZE * MAX_SIZE, 1])
    
    # Predictions
    # TODO: Possibly need to change
#     pred_classes = logits
    pred_classes = tf.map_fn(convertPredToRank, logits)
#     pred_classes = tf.argmax(logits, axis=1)
#     pred_probas = tf.nn.softmax(logits)
    
    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)
    
    # Define loss and optimizer
#     loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
#         logits=logits, labels=tf.cast(labels, dtype=tf.int32)))
#     loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
#         logits=logits, labels=labels))
#     loss_op = tf.reduce_mean(loss(labels, logits))
    loss_op = tf.reduce_mean(listNetLoss(labels, logits))
#     loss_map = tf.map_fn(lambda x: listMLE2(x[0], x[1]), (labels, pred_classes), dtype=tf.float64)
#     print(labels.get_shape()[0])
#     labels_length = tf.shape(labels)[0]
#     loss_op = tf.reduce_mean(listMLE2Loss(labels, logits, labels_length, tf.cast(labels_length, dtype=tf.float32)))
#     optimizer = tf.train.GradientDescentOptimizer(learning_rate=LEARNING_RATE)
    optimizer = tf.contrib.opt.NadamOptimizer(learning_rate=LEARNING_RATE)
#     optimizer = tf.train.AdamOptimizer()
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # Evaluate the accuracy of the model
#     acc_op = tf.metrics.accuracy(labels=tf.argmax(labels, axis=1), predictions=pred_classes)
#     acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    ndcg_map = tf.map_fn(lambda x: ndcg(x[0], x[1]), (labels, pred_classes), dtype=tf.float32)
    ndcg_op = tf.metrics.mean(ndcg_map)
    top1_map = tf.map_fn(lambda x: top1Match(x[0], x[1]), (labels, pred_classes), dtype=tf.float32)
    top1_op = tf.metrics.mean(top1_map)
    top2_map = tf.map_fn(lambda x: top2Match(x[0], x[1]), (labels, pred_classes), dtype=tf.float32)
    top2_op = tf.metrics.mean(top2_map)
    spearman_map = tf.map_fn(lambda x: boundedSpearman(x[0], x[1]), (labels, pred_classes), dtype=tf.float32)
    acc_op = tf.metrics.mean(spearman_map)
    
    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_classes,
      loss=loss_op,
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op, 'ndcg': ndcg_op, 'top1Classification': top1_op, 'top2Classification': top2_op})

    return estim_specs

In [16]:
# Build the Estimator
config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.8
model = tf.estimator.Estimator(modelFunc, config=tf.contrib.learn.RunConfig(session_config=config, save_summary_steps=10000, log_step_count_steps=10000))

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f879b256860>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 10000, '_save_checkpoints_secs': 600, '_log_step_count_steps': 10000, '_session_config': , '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmpw6yl6kuw'}


In [17]:
trainResults = []
validResults = []

In [18]:
trainFunc = tf.estimator.inputs.numpy_input_fn(
    x={"input": inputsTrain.astype(np.float32)}, y=outputsTrain.astype(np.float32),
    batch_size=BATCH_SIZE, shuffle=False)

validFunc = tf.estimator.inputs.numpy_input_fn(
    x={"input": inputsValid.astype(np.float32)}, y=outputsValid.astype(np.float32),
    batch_size=BATCH_SIZE, shuffle=False)

# for i in range(0, 30):
model.train(inputFunc, steps=40000)

print("Evaluating Train")
accuracy = model.evaluate(trainFunc)
trainResults.append(accuracy)

print("Evaluating Valid")
accuracy = model.evaluate(validFunc)
validResults.append(accuracy)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpw6yl6kuw/model.ckpt.
INFO:tensorflow:loss = 0.322451, step = 1
INFO:tensorflow:loss = 0.313544, step = 101 (0.408 sec)
INFO:tensorflow:loss = 0.312324, step = 201 (0.356 sec)
INFO:tensorflow:loss = 0.312974, step = 301 (0.325 sec)
INFO:tensorflow:loss = 0.310869, step = 401 (0.311 sec)
INFO:tensorflow:loss = 0.313829, step = 501 (0.344 sec)
INFO:tensorflow:loss = 0.31318, step = 601 (0.372 sec)
INFO:tensorflow:loss = 0.313109, step = 701 (0.369 sec)
INFO:tensorflow:loss = 0.310563, step = 801 (0.331 sec)
INFO:tensorflow:loss = 0.306615, step = 901 (0.309 sec)
INFO:tensorflow:loss = 0.303172, step = 1001 (0.303 sec)
INFO:tensorflow:loss = 0.313593, step = 1101 (0.290 sec)
INFO:tensorflow:loss = 0.306733, step = 1201 (0.349 sec)
INFO:tensorflow:loss = 0.308235, step = 1301 (0.335 sec)
INFO:tensorflow:loss = 0.307532, step = 1401 (0.325 sec)
INFO:tensorflow:loss = 0.305438, step = 1501 (0.335

INFO:tensorflow:loss = 0.311544, step = 14101 (0.386 sec)
INFO:tensorflow:loss = 0.309289, step = 14201 (0.316 sec)
INFO:tensorflow:loss = 0.311305, step = 14301 (0.315 sec)
INFO:tensorflow:loss = 0.3086, step = 14401 (0.337 sec)
INFO:tensorflow:loss = 0.311243, step = 14501 (0.345 sec)
INFO:tensorflow:loss = 0.310322, step = 14601 (0.300 sec)
INFO:tensorflow:loss = 0.309059, step = 14701 (0.316 sec)
INFO:tensorflow:loss = 0.312463, step = 14801 (0.342 sec)
INFO:tensorflow:loss = 0.312357, step = 14901 (0.327 sec)
INFO:tensorflow:loss = 0.31423, step = 15001 (0.316 sec)
INFO:tensorflow:loss = 0.308423, step = 15101 (0.342 sec)
INFO:tensorflow:loss = 0.304514, step = 15201 (0.311 sec)
INFO:tensorflow:loss = 0.310272, step = 15301 (0.327 sec)
INFO:tensorflow:loss = 0.308538, step = 15401 (0.338 sec)
INFO:tensorflow:loss = 0.315187, step = 15501 (0.367 sec)
INFO:tensorflow:loss = 0.312574, step = 15601 (0.327 sec)
INFO:tensorflow:loss = 0.311155, step = 15701 (0.368 sec)
INFO:tensorflow:l

INFO:tensorflow:loss = 0.309915, step = 28201 (0.299 sec)
INFO:tensorflow:loss = 0.311232, step = 28301 (0.314 sec)
INFO:tensorflow:loss = 0.309469, step = 28401 (0.307 sec)
INFO:tensorflow:loss = 0.310897, step = 28501 (0.302 sec)
INFO:tensorflow:loss = 0.31004, step = 28601 (0.291 sec)
INFO:tensorflow:loss = 0.310071, step = 28701 (0.308 sec)
INFO:tensorflow:loss = 0.308543, step = 28801 (0.304 sec)
INFO:tensorflow:loss = 0.310645, step = 28901 (0.305 sec)
INFO:tensorflow:loss = 0.313239, step = 29001 (0.311 sec)
INFO:tensorflow:loss = 0.312887, step = 29101 (0.334 sec)
INFO:tensorflow:loss = 0.306438, step = 29201 (0.317 sec)
INFO:tensorflow:loss = 0.307577, step = 29301 (0.283 sec)
INFO:tensorflow:loss = 0.308632, step = 29401 (0.296 sec)
INFO:tensorflow:loss = 0.311016, step = 29501 (0.307 sec)
INFO:tensorflow:loss = 0.310268, step = 29601 (0.346 sec)
INFO:tensorflow:loss = 0.312311, step = 29701 (0.303 sec)
INFO:tensorflow:loss = 0.30969, step = 29801 (0.312 sec)
INFO:tensorflow:

In [20]:
predictions = list(model.predict(validFunc))

INFO:tensorflow:Restoring parameters from /tmp/tmpw6yl6kuw/model.ckpt-40000


In [21]:
import collections
collections.Counter(list(map(str, predictions)))

Counter({'[ 2.  1.  3.  4.  0.]': 2155})

In [22]:
collections.Counter(list(map(str, outputsValid)))

Counter({'[0 1 2 3 4]': 5,
         '[0 1 2 4 3]': 5,
         '[0 2 1 3 4]': 5,
         '[0 2 1 4 3]': 145,
         '[0 2 3 4 1]': 10,
         '[0 2 4 1 3]': 275,
         '[0 2 4 3 1]': 30,
         '[2 0 1 3 4]': 5,
         '[2 0 1 4 3]': 220,
         '[2 0 3 1 4]': 10,
         '[2 0 3 4 1]': 20,
         '[2 0 4 1 3]': 295,
         '[2 0 4 3 1]': 215,
         '[2 1 3 4 0]': 10,
         '[2 1 4 0 3]': 25,
         '[2 3 1 4 0]': 10,
         '[2 3 4 0 1]': 5,
         '[2 3 4 1 0]': 5,
         '[2 4 0 1 3]': 210,
         '[2 4 0 3 1]': 175,
         '[2 4 1 0 3]': 125,
         '[2 4 1 3 0]': 110,
         '[2 4 3 0 1]': 55,
         '[2 4 3 1 0]': 70,
         '[4 0 2 1 3]': 15,
         '[4 1 2 0 3]': 5,
         '[4 1 2 3 0]': 5,
         '[4 2 0 1 3]': 35,
         '[4 2 0 3 1]': 10,
         '[4 2 1 0 3]': 10,
         '[4 2 1 3 0]': 10,
         '[4 2 3 1 0]': 20,
         '[4 3 2 0 1]': 5})